# Exercise 3 - Range proofs with ZKPs

Now that we can do a privacy-preserving unlinkable selective disclosure, we can tackle the next step:
removing even more information by only proving the actually relevant information.
For an age check, the verifier doesn't need to know our birthdate.
It's enough if we can convince them that we are above a certain age.

## Sections

1. Issuer - Setting up and signing a credential
2. Creating a range proof using LegoGroth16
3. Creating a range proof using Bulletproofs++
4. Discussion: are we anonymous and unlinkable now?
5. Coding exercise - Add an accumlator using a negative membership check

---

## 1. Issuer - Setting up and signing a credential

In [ ]:
const E_ID_SCHEMA = {
    type: 'object',
    properties: {
        name: { type: 'string' },
        profession: { type: 'string' },
        timeOfBirth: { type: 'integer', minimum: 0, maximum: 120},
        height: {type: 'integer', minimum: 130, maximum: 210},
        weight: {type: 'number', minimum: 40, multipleOf: 0.1}
    }
}
const E_ID_DATA = {
    name: "Jack Sparrow",
    timeOfBirth: new Date("1993-08-01T00:00:00").getTime(),
    profession: "IT Manager",
    height: 176,
    weight: 84
}

In [ ]:
import { initializeWasm } from '@docknetwork/crypto-wasm-ts'
await initializeWasm();

// Generating a keypair once signature parameters are created.
import { BBSKeypair, BBSSignatureParams, BBS_SIGNATURE_PARAMS_LABEL_BYTES } from '@docknetwork/crypto-wasm-ts';

const stringToBytes = (str: string) => Uint8Array.from(Buffer.from(str, "utf-8"));

const params = BBSSignatureParams.generate(100, BBS_SIGNATURE_PARAMS_LABEL_BYTES);
const keypair = BBSKeypair.generate(params, stringToBytes('seed1'));

const secretKey = keypair.secretKey;
const publicKey = keypair.publicKey;

In [ ]:
// Issing a credential based on the E-ID data based on the specified schema
import { CredentialSchema, BBSCredentialBuilder, BBSCredential, SUBJECT_STR } from '@docknetwork/crypto-wasm-ts'

const baseSchema = CredentialSchema.essential(); // default skeleton for the schema
baseSchema.properties[SUBJECT_STR] = E_ID_SCHEMA;  // we insert our E-ID schema into the data schema position
let eIDSchema = new CredentialSchema(baseSchema);

const builder = new BBSCredentialBuilder();
builder.schema = eIDSchema;
builder.subject = E_ID_DATA;  // assigning data to the schema

const credential = builder.sign(secretKey);

const credentialToSendToHolder = credential.toJSON();

In [ ]:
credentialToSendToHolder

---

## 2. Creating a range proof using LegoGroth16

Our first range proof is done using [LegoGroth16](https://eprint.iacr.org/2019/142)

In [ ]:
// This section establishes a range proof capability based on the LegoGroth16 algorithm

// The verifier is responsible for creating these keys, then sharing only the proving key with 
// the credential holder
import { BoundCheckSnarkSetup, SetupParam } from '@docknetwork/crypto-wasm-ts';
const provingKey = BoundCheckSnarkSetup();
const snarkProvingKey = provingKey.decompress();
const snarkVerifyingKey = provingKey.getVerifyingKeyUncompressed();

## Credential Holder

In [ ]:
import { PresentationBuilder } from '@docknetwork/crypto-wasm-ts'
const builder = new PresentationBuilder();
builder.addCredential(credential, publicKey)
builder.markAttributesRevealed(0, new Set<string>(['credentialSubject.name', 'credentialSubject.profession']));
builder.enforceBounds(0, 'credentialSubject.height', 170, 190, 'ageRangeCheck', snarkProvingKey); // The range is inclusive of the minimum and exclusive of the maximum value

const presentation = builder.finalize();

### sending data to Verifier
data: 
1. selected data to be revealed
2. Schema of the credential
3. Proof of signature

All these info and more are stored in the "presentation object". Note that this object is a completely serializable JSON object.

## Verifier

In [ ]:
const receivedObjectOnVerifierSide = presentation.toJSON()
console.log(receivedObjectOnVerifierSide)  // This is a serialized version (to be sent across an API to the verifier).
console.log("-----------------------------------")
console.log(receivedObjectOnVerifierSide["spec"]["credentials"][0])

In [ ]:
import { Presentation } from '@docknetwork/crypto-wasm-ts'

const predicateParams = new Map([['ageRangeCheck', snarkVerifyingKey]]);

const recreatedPres = Presentation.fromJSON(receivedObjectOnVerifierSide)
recreatedPres.verify([publicKey], undefined, predicateParams)

### Now, we redo the experiment with Bulletproofs++
reference: https://eprint.iacr.org/2022/510

In [ ]:
import { BoundCheckBppParams } from '@docknetwork/crypto-wasm-ts'

## Holder

In [ ]:
import { PresentationBuilder } from '@docknetwork/crypto-wasm-ts'
const builder = new PresentationBuilder();
builder.addCredential(credential, publicKey)
builder.markAttributesRevealed(0, new Set<string>(['credentialSubject.name', 'credentialSubject.profession']));

// The Holder creates his own parameters for the range proof
const boundCheckBppParams1 = new BoundCheckBppParams(stringToBytes('Common Reference String')).decompress();
// builder.enforceBounds(0, 'credentialSubject.timeOfBirth', new Date("1972-01-01T00:00:00").getTime(), new Date("2020-08-01T00:00:00").getTime(), 'ageRangeCheck', boundCheckBppParams1);
builder.enforceBounds(0, 'credentialSubject.height', 170, 190, 'ageRangeCheck', boundCheckBppParams1);

const presentation = builder.finalize();
const receivedObjectOnVerifierSide = presentation.toJSON()
presentation.spec.credentials[0]

## Verifier

In [ ]:
import { Presentation } from '@docknetwork/crypto-wasm-ts'

// The verifier creates his own parameters for the range proof
const boundCheckBppParams2 = new BoundCheckBppParams(stringToBytes('Common Reference String')).decompress();

const predicateParams1 = new Map([['ageRangeCheck', boundCheckBppParams2]]);

const recreatedPres = Presentation.fromJSON(receivedObjectOnVerifierSide)
recreatedPres.verify([publicKey], undefined, predicateParams1)

## Optional Exercise: Add an accumlator using a negative membership check  
Use the documentation to implement an accumlator using a negative membership check